# Exploration using Pandas

In [25]:
import pandas as pd
import os
import sys
sys.path.insert(1, os.path.abspath('../'))
import util

In [14]:
base_path = os.path.join(os.path.abspath(''), os.pardir, os.pardir)
base_path

'/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/src/test-scripts/../..'

In [17]:
save_filepath = 'file://' + os.path.abspath(os.path.join(base_path, 'data', 'delta'))
print(save_filepath)

file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/delta


In [18]:
temp_filepath = 'file://' + os.path.abspath(os.path.join(base_path, 'data', 'tmp'))
temp_filepath

'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/tmp'

In [2]:
base_url = 'https://datasets.imdbws.com'

files_list = [
        "name.basics.tsv.gz",
        "title.akas.tsv.gz",
        "title.basics.tsv.gz",
        "title.crew.tsv.gz",
        "title.episode.tsv.gz",
        "title.principals.tsv.gz",
        "title.ratings.tsv.gz"]
        
temp_filepath = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, 'data', 'tmp'))

In [3]:
data = dict()

for file in files_list:
    data[file] = pd.read_csv(temp_filepath+'/'+file, sep = '\t', nrows=100, compression='gzip',error_bad_lines=False)

In [11]:
data["title.akas.tsv.gz"].head(10)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1
5,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1
6,tt0000002,2,A bohóc és kutyái,HU,\N,imdbDisplay,\N,0
7,tt0000002,3,Le clown et ses chiens,FR,\N,\N,\N,0
8,tt0000002,4,Clovnul si cainii sai,RO,\N,imdbDisplay,\N,0
9,tt0000002,5,Клоун и его собаки,RU,\N,\N,\N,0


In [12]:
data["title.basics.tsv.gz"].head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [13]:
data["title.crew.tsv.gz"].head(10)

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
5,tt0000006,nm0005690,\N
6,tt0000007,"nm0005690,nm0374658",\N
7,tt0000008,nm0005690,\N
8,tt0000009,nm0085156,nm0085156
9,tt0000010,nm0525910,\N


In [15]:
data["title.episode.tsv.gz"].head(10)

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
5,tt0043693,tt0989125,2,8
6,tt0043710,tt0989125,3,3
7,tt0044093,tt0959862,1,6
8,tt0044901,tt0989125,3,46
9,tt0045519,tt0989125,4,11


In [7]:
data["title.principals.tsv.gz"].head(20)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm5442194,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


In [16]:
data["title.ratings.tsv.gz"].head(10)

,tconst,averageRating,numVotes
0,tt0000001,5.6,1529
1,tt0000002,6.1,185
2,tt0000003,6.5,1175
3,tt0000004,6.2,113
4,tt0000005,6.1,1892
5,tt0000006,5.2,102
6,tt0000007,5.4,609
7,tt0000008,5.4,1635
8,tt0000009,5.4,80
9,tt0000010,6.9,5453


# Exploration using PySpark

In [27]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

env: JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home


In [28]:
import configparser
import datetime
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import sys

sys.path.insert(1, os.path.abspath('../'))
import util

In [29]:
def create_spark_session(master,endpoint=None):
    spark = SparkSession \
            .builder \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,io.delta:delta-core_2.11:0.3.0") \
            .config("fs.s3a.endpoint",endpoint)\
            .config("spark.sql.autoBroadcastJoinThreshold",-1) \
            .appName("udacity-dend-capstone-etl-proj")\
            .master(master)\
            .getOrCreate()
    
    return spark

In [30]:
#spark://127.0.0.1:7077
spark = create_spark_session("spark://127.0.0.1:7077","s3.us-west-2.amazonaws.com")
spark

## Download files to local directory

In [5]:
base_url = 'https://datasets.imdbws.com'

files_list = [
        "name.basics.tsv.gz",
        "title.akas.tsv.gz",
        "title.basics.tsv.gz",
        "title.crew.tsv.gz",
        "title.episode.tsv.gz",
        "title.principals.tsv.gz",
        "title.ratings.tsv.gz"]
        
download_directory = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, 'data', 'tmp'))

In [11]:
util.download_files_to_local(base_url, files_list, download_directory)

## Process name.basics file

In [9]:
'''
names_schema = StructType([
                    StructField('nconst', StringType(), True),
                    StructField('primaryName', StringType(), True),
                    StructField('birthYear', IntegerType(), True),
                    StructField('deathYear', IntegerType(), True),
                    StructField('primaryProfession', StringType(), True),
                    StructField('knownForTitles', StringType(), True),
                    StructField('broken', StringType(), True)
                        ])
'''

temp_filepath = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/tmp'
file = 'name.basics.tsv.gz'

names_df_raw = spark.read.load(
    temp_filepath+'/'+file,
    format="csv", 
    sep="\t", 
    inferSchema="true", 
    header="true",
    ignoreLeadingWhiteSpace=True,
    ignoreTrailingWhiteSpace=True,
    nullValue = '\\N',
    quote = '' # this will ignore using quotes as a qualifier. This helps reduce malformed records. 
)

### Data Exploration

In [10]:
names_df_raw.printSchema()
names_df = names_df_raw.withColumnRenamed("nconst","artistId")
names_df.show(5)

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- deathYear: integer (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)

+---------+---------------+---------+---------+--------------------+--------------------+
| artistId|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0043044,tt00723...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt01170...|
|nm0000003|Brigitte Bardot|     1934|     null|actress,soundtrac...|tt0057345,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,soun...|tt0078723,tt00804...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050976,tt00839...|
+---------+---

In [11]:
names_df.filter(names_df.birthYear < 1000).count()
names_df.filter(names_df.deathYear < 1000).count()

# For very small number of cases, the birthYear and deathYear is less than 1000 (15 and 18 respectively)
# Spot checking a few entries, this mostly seems to be an error in the dataset
# Rather than removing these entries, marking these fields as null seems appropriate

15

In [24]:
def fix_year(col):
    # Get today's date
    now = datetime.datetime.now()
    
    fix_year_func = (
                F.when(F.col(col)<1000,None)
                 .when(F.col(col)>now.year,None)
                 .otherwise(F.col(col))
                )
    return fix_year_func


names_df = names_df.withColumn("birthYear_fixed",fix_year('birthYear'))\
            .drop("birthYear")\
            .withColumnRenamed("birthYear_fixed", "birthYear")

names_df = names_df.withColumn("deathYear_fixed",fix_year('deathYear'))\
             .drop("deathYear")\
             .withColumnRenamed("deathYear_fixed", "deathYear")

names_df.show(10, False)

+---------+---------------+------------------------------+---------------------------------------+---------+---------+
|artistId |primaryName    |primaryProfession             |knownForTitles                         |birthYear|deathYear|
+---------+---------------+------------------------------+---------------------------------------+---------+---------+
|nm0000001|Fred Astaire   |soundtrack,actor,miscellaneous|tt0043044,tt0072308,tt0053137,tt0050419|1899     |1987     |
|nm0000002|Lauren Bacall  |actress,soundtrack            |tt0071877,tt0117057,tt0038355,tt0037382|1924     |2014     |
|nm0000003|Brigitte Bardot|actress,soundtrack,producer   |tt0057345,tt0049189,tt0059956,tt0054452|1934     |null     |
|nm0000004|John Belushi   |actor,writer,soundtrack       |tt0078723,tt0080455,tt0077975,tt0072562|1949     |1982     |
|nm0000005|Ingmar Bergman |writer,director,actor         |tt0050976,tt0083922,tt0069467,tt0050986|1918     |2007     |
|nm0000006|Ingrid Bergman |actress,soundtrack,pr

In [25]:
artists_df = names_df.select("artistId","primaryName","birthYear","deathYear")
artists_prmry_prfsn_df = names_df.select("artistId",\
                                         F.explode(F.split(F.col("primaryProfession"),",")).alias("primaryProfession"))
artists_knwn_fr_ttls_df = names_df.select("artistId",
                                          F.explode(F.split(F.col("knownForTitles"),",")).alias("knownForTitles"))

In [26]:
artists_df.show(5)
artists_prmry_prfsn_df.show(5)
artists_knwn_fr_ttls_df.show(5)

+---------+---------------+---------+---------+
| artistId|    primaryName|birthYear|deathYear|
+---------+---------------+---------+---------+
|nm0000001|   Fred Astaire|     1899|     1987|
|nm0000002|  Lauren Bacall|     1924|     2014|
|nm0000003|Brigitte Bardot|     1934|     null|
|nm0000004|   John Belushi|     1949|     1982|
|nm0000005| Ingmar Bergman|     1918|     2007|
+---------+---------------+---------+---------+
only showing top 5 rows

+---------+-----------------+
| artistId|primaryProfession|
+---------+-----------------+
|nm0000001|       soundtrack|
|nm0000001|            actor|
|nm0000001|    miscellaneous|
|nm0000002|          actress|
|nm0000002|       soundtrack|
+---------+-----------------+
only showing top 5 rows

+---------+--------------+
| artistId|knownForTitles|
+---------+--------------+
|nm0000001|     tt0043044|
|nm0000001|     tt0072308|
|nm0000001|     tt0053137|
|nm0000001|     tt0050419|
|nm0000002|     tt0071877|
+---------+--------------+
only 

In [27]:
artists_df.describe().show(5)

+-------+---------+--------------------+------------------+------------------+
|summary| artistId|         primaryName|         birthYear|         deathYear|
+-------+---------+--------------------+------------------+------------------+
|  count|  9537755|             9537755|            486464|            169619|
|   mean|     null|            Infinity|1950.7003293152218|1988.4233900683296|
| stddev|     null|                 NaN| 33.18508746296023|29.796695760170913|
|    min|nm0000001|!'aru Ikhuisi Pie...|              1048|              1022|
|    max|nm9993719|þórunn Ósk Morinó...|              2018|              2019|
+-------+---------+--------------------+------------------+------------------+



### Write dataframes to Parquet

In [28]:
save_file_path = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/'

artists_df.write.mode('overwrite').parquet(save_file_path+"artists.parquet")
artists_prmry_prfsn_df.write.mode('overwrite').parquet(save_file_path+"artists_prmry_profession.parquet")
artists_knwn_fr_ttls_df.write.mode('overwrite').parquet(save_file_path+"artists_knwnfor_titles.parquet")

# Process title.basics file

In [8]:
temp_filepath = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/tmp'
file = 'title.basics.tsv.gz'

title_basics_df_raw = spark.read.load(
    temp_filepath+'/'+file,
    format="csv", 
    sep="\t", 
    inferSchema="true", 
    header="true",
    ignoreLeadingWhiteSpace=True,
    ignoreTrailingWhiteSpace=True,
    nullValue = '\\N',
    quote = '' # this will ignore using quotes as a qualifier. This helps reduce malformed records. 
)

### Data Exploration

In [9]:
title_basics_df_raw.printSchema()
title_basics_df = title_basics_df_raw.withColumnRenamed("tconst","titleId")
title_basics_df.show(5)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: string (nullable = true)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|  titleId|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animat

In [40]:
title_basics_df.describe("startYear","endYear","runtimeMinutes").show()

+-------+------------------+------------------+------------------+
|summary|         startYear|           endYear|    runtimeMinutes|
+-------+------------------+------------------+------------------+
|  count|           5774481|             52199|           1819184|
|   mean|2001.1566693179873|2001.4973658499205| 45.22753003544446|
| stddev|21.304072129841646|16.874377455258745|120.73609491107416|
|    min|              1874|              1924|                 0|
|    max|              2115|              2027|            125156|
+-------+------------------+------------------+------------------+



In [10]:
titles_df = title_basics_df.select("titleId","titleType","primaryTitle","originalTitle",\
                                   "isAdult","startYear","endYear","runtimeMinutes")

titles_genres_df = title_basics_df.select("titleId",F.explode(F.split(F.col("genres"),",")).alias("genres"))

In [42]:
titles_df.show(5)
titles_genres_df.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+
|  titleId|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|          null|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   null|             1|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+
only showing top 5 rows

+---------+-----------+
|  tit

### Write dataframes to Parquet


In [11]:
save_file_path = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/delta/'

titles_df.write.format("delta").mode('overwrite').partitionBy("startYear").save(save_file_path+"titles")

In [43]:
save_file_path = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/'

titles_df.write.mode('overwrite').partitionBy("startYear").parquet(save_file_path+"titles.parquet")
titles_genres_df.write.mode('overwrite').parquet(save_file_path+"titles_genres.parquet")

# Process title.ratings file

In [7]:
temp_filepath = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/tmp'
file = 'title.ratings.tsv.gz'

title_ratings_df_raw = spark.read.load(
    temp_filepath+'/'+file,
    format="csv", 
    sep="\t", 
    inferSchema="true", 
    header="true",
    ignoreLeadingWhiteSpace=True,
    ignoreTrailingWhiteSpace=True,
    nullValue = '\\N',
    quote = '' # this will ignore using quotes as a qualifier. This helps reduce malformed records. 
)

In [8]:
title_ratings_df_raw.printSchema()
title_ratings_df = title_ratings_df_raw.withColumnRenamed("tconst","titleId")
title_ratings_df.show(5)

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)

+---------+-------------+--------+
|  titleId|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.6|    1529|
|tt0000002|          6.1|     185|
|tt0000003|          6.5|    1175|
|tt0000004|          6.2|     113|
|tt0000005|          6.1|    1892|
+---------+-------------+--------+
only showing top 5 rows



In [9]:
save_file_path = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/'

title_ratings_df.write.mode('overwrite').parquet(save_file_path+"title_ratings.parquet")

# Joining titles, title_ratings and title_genres datasets

In [31]:
title_ratings_filepath = "file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/title_ratings.parquet"
titles_filepath = "file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/titles.parquet"
title_genres_filepath = "file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/titles_genres.parquet"

title_ratings_df = spark.read.parquet(title_ratings_filepath)
titles_df = spark.read.parquet(titles_filepath)
title_genres_df = spark.read.parquet(title_genres_filepath)

In [32]:
title_genres_ratings_df = titles_df.join(title_ratings_df, "titleId").join(title_genres_df,"titleId")
title_genres_ratings_df = title_genres_ratings_df\
                            .groupBy("startYear","genres")\
                            .agg({"averageRating":"mean","numVotes":"sum","titleId":"count"})\
                            .withColumnRenamed("avg(averageRating)","averageRating")\
                            .withColumnRenamed("sum(numVotes)","numVotes")\
                            .withColumnRenamed("count(titleId)","numTitles")

In [33]:
title_ratings_df.count()

965853

In [34]:
titles_df.join(title_ratings_df, "titleId").count()

965850

In [35]:
title_genres_df.count()

9284639

In [36]:
titles_df.join(title_genres_df,"titleId").count()

9284639

In [37]:
title_genres_ratings_df.count()

3102

In [38]:
title_ratings_sql_df = title_ratings_df.createOrReplaceTempView("title_ratings_df")
titles_sql_df = titles_df.createOrReplaceTempView("titles")
title_genres_sql_df = title_genres_df.createOrReplaceTempView("title_genres")

In [39]:
spark.sql('''
select count(distinct titleId)
from title_genres
''').show()

spark.sql('''
select count(distinct titleId)
from titles
''').show()

spark.sql('''
select count(distinct titleId)
from title_ratings_df
''').show()

+-----------------------+
|count(DISTINCT titleId)|
+-----------------------+
|                5631946|
+-----------------------+

+-----------------------+
|count(DISTINCT titleId)|
+-----------------------+
|                6117678|
+-----------------------+

+-----------------------+
|count(DISTINCT titleId)|
+-----------------------+
|                 965853|
+-----------------------+



In [48]:
spark.sql('''
select tr.*
from title_ratings_df tr
left join titles t on tr.titleId = t.titleId
where t.titleId is null
''').show()

+---------+-------------+--------+
|  titleId|averageRating|numVotes|
+---------+-------------+--------+
|tt7820738|          7.9|      18|
|tt5607886|          8.6|       7|
|tt9315000|          5.8|     181|
+---------+-------------+--------+



In [44]:
title_genres_ratings_df = spark.sql('''
select tr.titleId, tr.averageRating, tr.numVotes, tg.genres, t.startYear
from title_ratings_df tr
inner join title_genres tg on tr.titleId = tg.titleId
inner join titles t on tg.titleId = t.titleId
''')

In [45]:
title_genres_ratings_df.count()

1860006

In [46]:
title_genres_ratings_df = title_genres_ratings_df\
                            .groupBy("startYear","genres")\
                            .agg({"averageRating":"mean","numVotes":"sum","titleId":"count"})\
                            .withColumnRenamed("avg(averageRating)","averageRating")\
                            .withColumnRenamed("sum(numVotes)","numVotes")\
                            .withColumnRenamed("count(titleId)","numTitles")

In [47]:
title_genres_ratings_df.count()

3102

In [29]:
save_file_path = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/'

title_genres_ratings\
    .write.mode('overwrite')\
    .partitionBy("startYear")\
    .parquet(save_file_path+"title_genres_ratings.parquet")

# Join title and title_ratings to get top 3 movies by year

In [35]:
base_path = os.path.join(os.path.abspath(''), os.pardir, os.pardir)

save_filepath = os.path.abspath(os.path.join(base_path, 'data', 'delta'))

temp_filepath = os.path.abspath(os.path.join(base_path, 'data', 'tmp'))

In [36]:
title_df = spark.read.format("delta").load(os.path.join(save_filepath,'titles'))
title_ratings_df = spark.read.format("delta").load(os.path.join(save_filepath,'title_ratings'))

In [40]:
title_df.createOrReplaceTempView("titles")
title_ratings_df.createOrReplaceTempView("title_ratings")

In [41]:
title_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)



In [42]:
title_ratings_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [39]:
spark.sql('''
select distinct titleType
from titles
''').show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [46]:
top_5_movies_by_year_df = spark.sql('''
select 
    titleId, 
    titleType,
    startYear,
    primaryTitle,
    averageRating,
    numVotes,
    rank
from (
    select 
        t.titleId, 
        t.titleType,
        t.startYear,
        t.primaryTitle,
        tr.averageRating,
        tr.numVotes,
        rank() over (partition by t.titleType, t.startYear order by averageRating desc) as rank
    from titles t
    inner join title_ratings tr 
        on t.titleId = tr.titleId
    where t.titleType in ('movie', 'tvMovie')) tmp
where rank < 6
''')